# Show Commands Script

During this lab we are going to utilize Netmiko to log into a handful of routers and issue some basic show commands. 

This lab will also utilize three methods for returning the data:
standard: String
Genie: This utilizes the Cisco Genie library and will return the data as a dictionary.
TextFSM: This utilizes the Networktocode templates and will return the data as a list of dictionaries.

### Import Statements

In order for this script to run we are going to need some specific libraries:
yaml = read in our inventory file.
getpass = securely prompt user for the password.
netmiko = handle logging into the equipment and execute commands
pprint = pretty printing of the output.

In [1]:
import yaml
from getpass import getpass
from netmiko import Netmiko
from pprint import pprint

### Read Inventory

This is a basic function that will read in a yaml file and return it as a dictionary.

The YAML format should be in:
```
device_name:  
  host: IP address of host
  port: SSH port
  device_type: Associated device type of the device
```

In [2]:
def read_inventory(inventory_file):
    with open(inventory_file, 'r') as file:
        hosts = yaml.safe_load(file)
        return hosts

### Display output using Genie

Genie is 

In [3]:
def show_commands_genie(session, commands):
    for command in commands:
        output = session.send_command(command, use_genie=True)
        pprint(output) 
        pprint(type(output))

### Display output using TextFSM


In [4]:
def show_commands_textfsm(session, commands):
    for command in commands:
        output = session.send_command(command, use_textfsm=True)
        pprint(output) 
        pprint(type(output))

### Standard output display


In [5]:
def show_commands_standard(session, commands):
    for command in commands:
        output = session.send_command(command)
        pprint(output) 
        pprint(type(output))

In [6]:
def main():
    inventory_file = 'inventory/routers.yml'
    
    # List of show commands
    commands = ['show ip interface brief']
    
    # Prompt for username and password
    username = input('Enter the username: ')
    password = getpass('Enter the password: ')
    
    # Read in the inventory
    hosts = read_inventory(inventory_file)
    
    # Loop through the inventory and use a context handler for the Netmiko session
    for key, value in hosts.items():
        device = {'host': value['host'], 'port': value['port'],
                  'username': username, 'password': password,
                  'device_type': value['device_type']}
        with Netmiko(**device) as session:
            print(f"Now runing for {key}")
            # Uncomment to display output using Genie
            # show_commands_genie(session, commands)
            
            # Uncomment to display output using TextFSM
            # show_commands_textfsm(session, commands)
            
            # Uncomment to use standard output display
            show_commands_standard(session, commands)
            
            print('-'*100)

In [7]:
if __name__ == '__main__':
    main()

Enter the username: admin
Enter the password: ········
Now runing for csr1
('Interface              IP-Address      OK? Method Status                '
 'Protocol\n'
 'GigabitEthernet1       172.16.1.10     YES NVRAM  up                    '
 'up      \n'
 'GigabitEthernet2       192.168.100.1   YES NVRAM  up                    '
 'up      \n'
 'GigabitEthernet3       unassigned      YES NVRAM  administratively down '
 'down    \n'
 'GigabitEthernet4       unassigned      YES NVRAM  up                    '
 'up      \n'
 'Loopback1              1.1.1.1         YES NVRAM  up                    '
 'up      ')
<class 'str'>
----------------------------------------------------------------------------------------------------
Now runing for csr3
('Interface              IP-Address      OK? Method Status                '
 'Protocol\n'
 'GigabitEthernet1       172.16.1.13     YES NVRAM  up                    '
 'up      \n'
 'GigabitEthernet2       192.168.100.2   YES NVRAM  up                 